# **Importing SpaCy.** 

In [ ]:
import spacy
unlp = spacy.blank('ur')

# **Loading the corpus.**

Note : Please upload the files before executing the code.

In [ ]:
#Uploaded the three text files manually.
faiz_corpus = open('faiz.txt').read()
ghalib_corpus = open('ghalib.txt').read()
iqbal_corpus = open('iqbal.txt').read()
total_corpus = faiz_corpus + ghalib_corpus + iqbal_corpus

In [ ]:
# faiz_text = unlp(faiz_corpus)
# ghalib_text = unlp(ghalib_corpus)
# iqbal_text = unlp(iqbal_corpus)

# **Generating sentences**

In [ ]:
faiz_sentences = faiz_corpus.splitlines()
ghalib_sentences = ghalib_corpus.splitlines()
iqbal_sentences = iqbal_corpus.splitlines()

Sentences are separated based on spaces. After that, we removed any unnecessary punctuation as can be seen below and make a list of sentences.

In [ ]:
# punctuation = "؟!‘’:“''۔،"
punctuation = ['،','۔',"''",'“',':','’','‘','!','؟',')','(']
faiz_sentences = [sentence if sentence != ' ' else None for sentence in faiz_sentences]
faiz_sentences = list(filter(None,faiz_sentences))
for i in punctuation:
  faiz_sentences = [sentence.replace(i, '') for sentence in faiz_sentences]

ghalib_sentences = [sentence if sentence != ' ' else None for sentence in ghalib_sentences]
ghalib_sentences = list(filter(None,ghalib_sentences))
for i in punctuation:
  ghalib_sentences = [sentence.replace(i, '') for sentence in ghalib_sentences]

iqbal_sentences = [sentence if sentence != ' ' else None for sentence in iqbal_sentences]
iqbal_sentences = list(filter(None,iqbal_sentences))
for i in punctuation:
  iqbal_sentences = [sentence.replace(i, '') for sentence in iqbal_sentences]

print("Number of Faiz Sentences : ",len(faiz_sentences))
print("Number of Ghalib Sentences : ",len(ghalib_sentences))
print("Number of Iqbal Sentences : ",len(iqbal_sentences))
total_sentences = faiz_sentences + ghalib_sentences + iqbal_sentences

# **Generating words.**

We separate each word based on the spaces and generate a list of words of one sentence. "total_words" is a list of sentences, and each list of sentences has a list of words inside it. 

In [ ]:
faiz_words = list()
ghalib_words = list()
iqbal_words = list()

for word in faiz_sentences:
  faiz_words.append(word.split(' '))
for word in ghalib_sentences:
  ghalib_words.append(word.split(' '))
for word in iqbal_sentences:
  iqbal_words.append(word.split(' '))
flat_list_faiz = [item for sublist in faiz_words for item in sublist]
flat_list_ghalib = [item for sublist in ghalib_words for item in sublist]
flat_list_iqbal = [item for sublist in iqbal_words for item in sublist]
print("Number of words in Faiz : ", len(flat_list_faiz))
print("Number of words in Ghalib : ", len(flat_list_ghalib))
print("Number of words in Iqbal : ", len(flat_list_iqbal))
flat_list_words = len(flat_list_faiz) + len(flat_list_ghalib) + len(flat_list_iqbal) 
total_words = faiz_words + ghalib_words + iqbal_words

# **Generating Unigram Model**

For this task, I have firstly taken out the number of occurrences of each word separately and stored that in a dictionary called "wordOccurences".

In [ ]:
wordOccurences = dict()
for sentence in total_words:
  for word in sentence:
    if word not in wordOccurences:
      wordOccurences[word] = 1
    else:
      wordOccurences[word] += 1
tempWO = sorted(wordOccurences,key=wordOccurences.get,reverse = True)
# totalWords = 0
# for i in tempWO: 
#   print("Word : ", i, " occurences : " , wordOccurences[i])
#   totalWords += wordOccurences[i]
# print("total words : ", totalWords)

Next I have divided the occurences of each word with the number of total words in the dataset, because if I instead used the conditional probability formula, I would've always gotten '1.0' as the probability.

In [ ]:
unigrams = dict()
for key, value in wordOccurences.items():
  unigrams[key] = (value / flat_list_words)
sortedUnigrams = sorted(unigrams,key=unigrams.get,reverse = True)
# for i in tempUnigrams:
#  print("Word : ", i, " probability : " , unigrams[i])

I have found the list of first and last words too here.

In [ ]:
import random
first_words = list()
last_words = list()
for sentence in total_words:
  first_words.append(sentence[0])
  last_words.append(sentence[len(sentence)-1])

Here I generate the Stanzas. I select the first word randomly from the list of first words and then I choose subsequent words randomly from the list of unigrams. This is completely random and the verses don't make any actual sense hence this model is not very effective.

In [ ]:
def GenerateUnigramStanza():
  stanzaList = list()
  # print("Length : ", length)
  for stanzaCount in range(0,3):
    stanza = list()
    for i in range(0,4):
      length = random.randint(7,10)
      firstWord = random.choice(first_words)
      # print("First word : ", firstWord)
      verse = str()
      verse = verse + firstWord
      verse = verse + " "
      for j in range(1,length):
        word = random.choice(list(unigrams.keys()))
        # print("random word : ", word)
        verse = verse + word
        verse = verse + " "
      stanza.append(verse)
    stanzaList.append(stanza)
    stanzaList.append("\n") 
  return stanzaList

unigramStanzas = GenerateUnigramStanza()
for stanza in unigramStanzas:
  for verse in stanza: 
    print(verse)

# **Generating Bigram Model**

Firstly I find all the bigrams.

In [ ]:
bigramsList = list()
for sentence in total_words:
  for i in range(len(sentence)-1):
    w1 = sentence[i]
    w2 = sentence[i+1]
    bigramsList.append(tuple((w1,w2)))
# for x in bigramsList:
#   print(x[0],x[1])

Then I take out all the occurrences of each bigram respectively and store it in a dictionary called "bigramsOccurrences".

In [ ]:
bigramsOccurrences = dict()
for bigram in bigramsList:
    if bigram not in bigramsOccurrences:
      bigramsOccurrences[bigram] = 1
    else:
      bigramsOccurrences[bigram] += 1
# bigramsOccurrences

For each bigram, I find out it's conditional probability over here and sort them on the basis of their probabilities in a list.

In [ ]:
bigrams = dict()
for key, value in bigramsOccurrences.items(): # value = count(w1,w2)
  denom = wordOccurences[key[0]]
  bigrams[key] = (value / denom)
  # print(key[0],key[1])
sortedBigrams = sorted(bigrams,key=bigrams.get,reverse = True)
# for i in sortedBigrams:
#   print("Bigram : ", i, " probability : " , bigrams[i])

Here I generate the actual stanzas. For this, firstly I pick a random starting word for the sentences from the list of first words. Then I use that word as the context to generate the next word, and then the next word to generate the third word and so on... Thus using each newly generated word to generate the word after it. In the case where no bigram exists for the preceding word i.e. the preceding word was a 'last word' then we randomly select any word from the top 10 unigrams. This is by far a better method than Unigrams as the verses generated make some sense compared to the completely random verses generated by unigrams that often do not make any sense at all.

In [ ]:
def getPredictedNextWords(word) -> list:
  predictedWords = dict()
  for key,value in bigrams.items():
    if key[0] == word:
      predictedWords[key[1]] = value
  sortedWords = sorted(predictedWords,key=predictedWords.get,reverse = True)
  if len(sortedWords) > 10:
    sortedWords = sortedWords[:10]
  if len(sortedWords) == 0:#If no bigram found i.e last word.
    sortedWords = sortedUnigrams[:10]
  return sortedWords

def GenerateBigramStanza(): 
  stanzaList = list()
  # print("Length : ", length)
  for stanzaCount in range(0,3):
    stanza = list()
    for i in range(0,4):
      length = random.randint(7,10)
      firstWord = random.choice(first_words)
      # print("First word : ", firstWord)
      verse = str()
      verse = verse + firstWord
      verse = verse + " "
      tempWord = firstWord
      for j in range(1,length):
        word = random.choice(getPredictedNextWords(tempWord))
        verse = verse + word
        verse = verse + " "
        tempWord = word
        # print("Temp word : ", word)
      stanza.append(verse)
    stanzaList.append(stanza)
    stanzaList.append("\n") 
  return stanzaList

bigramStanzas = GenerateBigramStanza()
for stanza in bigramStanzas:
  for verse in stanza: 
    print(verse)

# **Generating Trigram Model**

Firstly we find all the trigrams.

In [ ]:
trigramsList = list()
for sentence in total_words:
  for i in range(len(sentence)-2):
    w1 = sentence[i]
    w2 = sentence[i+1]
    w3 = sentence[i+2]
    trigramsList.append(tuple((w1, w2, w3)))
  
# for x in trigramsList[:20]:
#   print(x[0], x[1], x[2])

Then we find each trigram's occurrences respectively and save it into a dictionary.

In [ ]:
trigramsOccurrences = dict()
for trigram in trigramsList:
    if trigram not in trigramsOccurrences:
      trigramsOccurrences[trigram] = 1
    else:
      trigramsOccurrences[trigram] += 1
# trigramsOccurrences

Next we calculate the conditional probability by finding the count of the bigram within the trigram and save this information in a dictionary and sort it.

In [ ]:
trigrams = dict()
for key, value in trigramsOccurrences.items(): # Value = count(w1,w1,w3)
  denom = bigramsOccurrences[key[:2]]
  trigrams[key] = (value / denom)
  # print(key[0],key[1])
sortedTrigrams = sorted(trigrams,key=trigrams.get,reverse = True)
# for i in sortedTrigrams:
#   print("Trigram : ", i, " probability : " , trigrams[i])

Here we will generate the actual stanzas. We select the first word randomly from the list of first words and get the second word from the list of bigrams randomly. Then on the basis of these two words, we generate the third word of the trigram randomly from the possibilities. After doing this, we use the second word and the newly generated word to generate the next word and this process carries on... Until all the subsequent words for the verse have been found. In the case where no third word is found for the preceding words i.e. the last two words are part of the end of the sentence, we randomly select a word from the top 10 unigrams. This is the most effective way thus far as most of the verses make actual sense and the context of the verses can be understood.

In [ ]:
def getTrigramsPredictedNextWords(w1,w2) -> list:
  predictedWords = dict()
  for key,value in trigrams.items():
    if key[0] == w1 and key[1] == w2:
      predictedWords[key[2]] = value
  sortedWords = sorted(predictedWords,key=predictedWords.get,reverse = True)
  if len(sortedWords) > 10:
    sortedWords = sortedWords[:10]
  if len(sortedWords) == 0:#If no trigram found i.e last word.
    sortedWords = sortedUnigrams[:10]
  return sortedWords

def GenerateTrigramStanza(): 
  stanzaList = list()
  # print("Length : ", length)
  for stanzaCount in range(0,3):
    stanza = list()
    for i in range(0,4):
      length = random.randint(7,10)
      firstWord = random.choice(first_words)
      secondWord = random.choice(getPredictedNextWords(firstWord))
      # print("First word : ", firstWord)
      verse = str()
      verse = verse + firstWord
      verse = verse + " "
      verse = verse + secondWord
      verse = verse + " "
      tempWord1 = firstWord
      tempWord2 = secondWord
      for j in range(2,length):
        word = random.choice(getTrigramsPredictedNextWords(tempWord1,tempWord2))
        verse = verse + word
        verse = verse + " "
        tempWord1 = tempWord2
        tempWord2 = word
        # print("Temp word : ", word)
      stanza.append(verse)
    stanzaList.append(stanza)
    stanzaList.append("\n") 
  return stanzaList


trigramStanzas = GenerateTrigramStanza()
for stanza in trigramStanzas:
  for verse in stanza: 
    print(verse)

# **Generating Backward Bigram Model**

We will use the same bigrams we generated before for backward bigram model. To generate Stanzas, instead of selecting a first word randomly, we will select a last word randomly. Using that last word, we will find the possible words preceding it and select any one randomly. We will repeat this process by using each randomly selected preceding word to select the subsequent preceding word until we have the whole verse. Lastly we will reverse the verse and append it to our stanza. This is pretty much the same thing we did for Bigram Model. The method is same, it's just that the direction of the context has been changed.

In [ ]:
def getBackwardBigramPredictedNextWord(word) -> list:
  predictedWords = dict()
  for key,value in bigrams.items():
    if key[1] == word:
      predictedWords[key[0]] = value
  sortedWords = sorted(predictedWords,key=predictedWords.get,reverse = True)
  if len(sortedWords) > 10:
    sortedWords = sortedWords[:10]
  if len(sortedWords) == 0:#If no bigram found i.e last word.
    sortedWords = sortedUnigrams[:10]
    print()
  return sortedWords

def GenerateBackwardBigramStanza(): 
  stanzaList = list()
  # print("Length : ", length)
  for stanzaCount in range(0,3):
    stanza = list()
    for i in range(0,4):
      length = random.randint(7,10)
      lastWord = random.choice(last_words)
      # print("Last word : ", lastWord)
      verse = str()
      verse = verse + lastWord
      verse = verse + " "
      tempWord = lastWord
      for j in range(1,length):
        word = random.choice(getPredictedNextWords(tempWord))
        verse = verse + word
        verse = verse + " "
        tempWord = word
        # print("Temp word : ", word)
      verseWords = verse.split()
      verseWords = list(reversed(verseWords))
      reversedVerse = str()
      for w in verseWords:
        reversedVerse = reversedVerse + w
        reversedVerse = reversedVerse + " "
      stanza.append(reversedVerse)
    stanzaList.append(stanza)
    stanzaList.append("\n") 
  return stanzaList


backwardBigramStanzas = GenerateBackwardBigramStanza()
for stanza in backwardBigramStanzas:
  for verse in stanza: 
    print(verse)

# **Generating Bidirectional Bigram Model**

We will use the same list of bigrams for this too. To generate stanzas, we will select a first word and a last word randomly. We will apply the same Bigram Model to the first word and the Backward Bigram Model to the last word. Thus we will generate our verse from the ends till the middle. Half of the verse will be generated by Backward Bigram Model and half from Bigram Model. This combines both the Bigram and Backward Bigram Models and is thus a combination of them both. No noticeable difference can be seen between between this or the Bigram Model and the Backward Bigram Model.

In [ ]:
def GenerateBidirectionalBigramStanza(): 
  stanzaList = list()
  # print("Length : ", length)
  for stanzaCount in range(0,3):
    stanza = list()
    for i in range(0,4):
      length = random.randint(7,10)
      firstWord = random.choice(first_words)
      lastWord = random.choice(last_words)
      # print("First word : ", firstWord)
      # print("Last word : ", lastWord)
      verse = str()
      verse1 = str()
      verse2 = str()
      verse1 = verse1 + firstWord
      verse1 = verse1 + " "
      verse2 = verse2 + lastWord
      verse2 = verse2 + " "
      tempWord1 = firstWord
      tempWord2 = lastWord
      if length%2 == 0: #Even length.
        # print("EVEN LENGTH")
        for j in range(1,length//2): 
          word = random.choice(getPredictedNextWords(tempWord1))
          verse1 = verse1 + word
          verse1 = verse1 + " "
          tempWord1 = word
        for k in range(1,length//2):
          word = random.choice(getBackwardBigramPredictedNextWord(tempWord2))
          verse2 = verse2 + word
          verse2 = verse2 + " "
          tempWord2 = word
      elif length%2 == 1:
        # print("ODD LENGTH")
        for j in range(1,(length//2) +1):
          word = random.choice(getPredictedNextWords(tempWord1))
          verse1 = verse1 + word
          verse1 = verse1 + " "
          tempWord1 = word
        for k in range(1,length//2):
          word = random.choice(getBackwardBigramPredictedNextWord(tempWord2))
          verse2 = verse2 + word
          verse2 = verse2 + " "
          tempWord2 = word
      verseWords = verse2.split()
      verseWords = list(reversed(verseWords))
      reversedVerse = str()
      for w in verseWords:
        reversedVerse = reversedVerse + w
        reversedVerse = reversedVerse + " "
      verse = verse1 + reversedVerse
      # print("Forward Verse : ", verse1)
      # print("Backward Verse : ", reversedVerse)
      stanza.append(verse)
    stanzaList.append(stanza)
    stanzaList.append("\n")
  return stanzaList

bidirectionalBigramStanzas = GenerateBidirectionalBigramStanza()
for stanza in bidirectionalBigramStanzas:
  for verse in stanza: 
    print(verse)